## Preprocess the data (as before)

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

C:\Users\mdebo\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mdebo\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mdebo\anaconda3\envs\PythonData\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mdebo\anaconda3\envs\PythonData\lib\site-packages

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
print(application_df.nunique())

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df.APPLICATION_TYPE.value_counts()

In [5]:
# Look at APPLICATION_TYPE value counts for binning
# use the variable name `application_types_to_replace`
application_types_to_replace = ('T9', 'T13', 'T12', 'T2', 'T14', 'T25', 'T15', 'T29', 'T17')

In [6]:
# Choose a cutoff value and create a list of application types to be replaced

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df.CLASSIFICATION.value_counts()

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
# greater_than_1 = (classification_count[classification_count > 1])

In [9]:
classifications_to_replace = list(classification_count[classification_count < 190].index)

In [10]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1003
C7000      777
C1700      287
C4000      194
Name: CLASSIFICATION, dtype: int64

In [11]:
ask_amt_labels = ['1-4999', '5000-9999', '10000-24999', '25000-99999', '100000-499999', '500000-999999', '1M-5M', '5M-10M', '10M-50M', '50M+']

ask_amt_bins = [0, 4999, 9999, 24999, 99999, 499999, 999999, 4999999, 9999999, 49999999,9999999999]

application_df['ASK_AMT'] = pd.cut(application_df['ASK_AMT'], bins=ask_amt_bins, labels=ask_amt_labels)

In [12]:
# Check to make sure binning was successful
application_df['ASK_AMT'].value_counts()

5000-9999        25946
25000-99999       2685
100000-499999     2305
10000-24999       1135
1M-5M              940
500000-999999      651
10M-50M            258
5M-10M             225
50M+               154
1-4999               0
Name: ASK_AMT, dtype: int64

In [13]:
# Set the value of ASK_AMT as object
application_df = application_df.astype({'ASK_AMT':'object'})

In [14]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            9
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
ASK_AMT                   9
dtype: int64

In [15]:
# Check the datatypes in the dataframe
application_df.dtypes

APPLICATION_TYPE          object
AFFILIATION               object
CLASSIFICATION            object
USE_CASE                  object
ORGANIZATION              object
STATUS                     int64
INCOME_AMT                object
SPECIAL_CONSIDERATIONS    object
ASK_AMT                   object
IS_SUCCESSFUL              int64
dtype: object

In [16]:
# Check the number of unique values in each column
application_df[application_cat].nunique()

APPLICATION_TYPE          9
AFFILIATION               6
CLASSIFICATION            9
USE_CASE                  5
ORGANIZATION              4
INCOME_AMT                9
SPECIAL_CONSIDERATIONS    2
ASK_AMT                   9
dtype: int64

In [17]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_10000-24999,ASK_AMT_100000-499999,ASK_AMT_10M-50M,ASK_AMT_1M-5M,ASK_AMT_25000-99999,ASK_AMT_5000-9999,ASK_AMT_500000-999999,ASK_AMT_50M+,ASK_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True, right_index=True)
application_df = application_df.drop(application_cat,1)
application_df.head()

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_10000-24999,ASK_AMT_100000-499999,ASK_AMT_10M-50M,ASK_AMT_1M-5M,ASK_AMT_25000-99999,ASK_AMT_5000-9999,ASK_AMT_500000-999999,ASK_AMT_50M+,ASK_AMT_5M-10M
0,1,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,1,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL.values
X = application_df.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=105)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Optimize the Model - Attempt #2

In [21]:
# # Dependencies
# import numpy as np
# import matplotlib.pyplot as plt

# # Example outlier plot of reaction times
# classifications = [17326,6074,4837,2261,1918,1883]
# fig1, ax1 = plt.subplots()
# ax1.set_title('classification types')
# ax1.set_ylabel('classification')
# ax1.boxplot(classifications)
# plt.show()

In [22]:
# # Determine which data points are outside of the 1.5*IQR range
# quartiles = np.quantile(classification,[.25,.75])
# iqr = quartiles[1]-quartiles[0]
# lower_bound = quartiles[0]-(1.5*iqr)
# upper_bound = quartiles[1]+(1.5*iqr)

# potential_outliers = [print(classification) if classification < lower_bound or classification > upper_bound else next for classification in classifications]

In [23]:
# potential_outliers

In [24]:
# # Look at CLASSIFICATION value counts for binning
# ask_amt_count = application_df.ASK_AMT.value_counts()
# ask_amt_count

In [25]:
# # You may find it helpful to look at ASK_AMT value counts >1
# greater_than_1_second = (ask_amt_count[ask_amt_count > 1])
# greater_than_1_second

In [26]:
# # You may find it helpful to look at ASK_AMT value counts >2
# greater_than_2 = (ask_amt_count[ask_amt_count > 2])
# greater_than_2

In [27]:
# # You may find it helpful to look at ASK_AMT value counts >2
# greater_than_3 = (ask_amt_count[ask_amt_count > 3])
# greater_than_3

In [28]:
# # Look at CLASSIFICATION value counts for binning
# ask_amount_count = application_df.ASK_AMT.value_counts()
# ask_amount_count

In [29]:
# ASK_AMT_1-4999
# ASK_AMT_5000-9999
# ASK_AMT_10000-24999
# ASK_AMT_25000-99999
# ASK_AMT_100000-499999
# ASK_AMT_500000-999999
# ASK_AMT_1M-5M
# ASK_AMT_5M-10M
# ASK_AMT_10M-50M


In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  7
hidden_nodes_layer2 = 9
hidden_nodes_layer3 = 1
hidden_nodes_layer4 = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 7)                 385       
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 10        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 2         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 2         
Total params: 471
Trainable params: 471
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.6936 - acc: 0.5343
Loss: 0.6936495932476167, Accuracy: 0.5343440175056458


In [33]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharityOptimized_Second06.h5")

In [34]:
# Add model weight checkpoints during training
import os
from tensorflow.keras.callbacks import ModelCheckpoint
os.makedirs("optimized_checkpoints_second/", exist_ok=True)
checkpoint_path="optimized_checkpoints_second/weights.{epoch}.hdf5"

# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq="epoch",
                              period=5)

In [35]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=[cp_callback])

Epoch 1/100
25724/25724 [==============================] - 1s 28us/sample - loss: 0.6831 - acc: 0.5318
Epoch 2/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.6605 - acc: 0.56870s - loss: 0.6639 - acc: 0.5
Epoch 3/100
25724/25724 [==============================] - 1s 22us/sample - loss: 0.6327 - acc: 0.7285
Epoch 4/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.6094 - acc: 0.7291
Epoch 5/100
25664/25724 [============================>.] - ETA: 0s - loss: 0.5941 - acc: 0.7305
Epoch 00005: saving model to optimized_checkpoints_second/weights.5.hdf5
25724/25724 [==============================] - 1s 22us/sample - loss: 0.5943 - acc: 0.7303
Epoch 6/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5855 - acc: 0.7310
Epoch 7/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5807 - acc: 0.7309
Epoch 8/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5781 - acc

Epoch 61/100
25724/25724 [==============================] - 1s 22us/sample - loss: 0.5629 - acc: 0.7352
Epoch 62/100
25724/25724 [==============================] - 1s 22us/sample - loss: 0.5630 - acc: 0.7348
Epoch 63/100
25724/25724 [==============================] - 1s 22us/sample - loss: 0.5631 - acc: 0.7338
Epoch 64/100
25724/25724 [==============================] - 1s 22us/sample - loss: 0.5631 - acc: 0.7345
Epoch 65/100
24032/25724 [===========================>..] - ETA: 0s - loss: 0.5617 - acc: 0.7359
Epoch 00065: saving model to optimized_checkpoints_second/weights.65.hdf5
25724/25724 [==============================] - 1s 22us/sample - loss: 0.5629 - acc: 0.7349
Epoch 66/100
25724/25724 [==============================] - 1s 22us/sample - loss: 0.5627 - acc: 0.7346
Epoch 67/100
25724/25724 [==============================] - 1s 21us/sample - loss: 0.5626 - acc: 0.7361
Epoch 68/100
25724/25724 [==============================] - 1s 22us/sample - loss: 0.5627 - acc: 0.7358
Epoch 69/1

In [36]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5690 - acc: 0.7298
Loss: 0.5690247993149493, Accuracy: 0.7297959327697754


In [37]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  7
hidden_nodes_layer2 = 9
hidden_nodes_layer3 = 1
hidden_nodes_layer4 = 1

#  YOUR CODE GOES HERE

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 7)                 385       
_________________________________________________________________
dense_6 (Dense)              (None, 9)                 72        
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 10        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 2         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 2         
Total params: 471
Trainable params: 471
Non-trainable params: 0
_________________________________________________________________


In [38]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=[cp_callback])

Epoch 1/100
25724/25724 [==============================] - 1s 27us/sample - loss: 0.6468 - acc: 0.6605
Epoch 2/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.6034 - acc: 0.7237
Epoch 3/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5874 - acc: 0.7267
Epoch 4/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5792 - acc: 0.7288
Epoch 5/100
24032/25724 [===========================>..] - ETA: 0s - loss: 0.5760 - acc: 0.7264
Epoch 00005: saving model to optimized_checkpoints_second/weights.5.hdf5
25724/25724 [==============================] - 1s 26us/sample - loss: 0.5749 - acc: 0.7275
Epoch 6/100
25724/25724 [==============================] - 1s 24us/sample - loss: 0.5727 - acc: 0.7292
Epoch 7/100
25724/25724 [==============================] - 1s 24us/sample - loss: 0.5712 - acc: 0.7299
Epoch 8/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5699 - acc: 0.7306
Epoch 9/100
25724/2

25724/25724 [==============================] - 1s 24us/sample - loss: 0.5500 - acc: 0.7355
Epoch 62/100
25724/25724 [==============================] - 1s 24us/sample - loss: 0.5497 - acc: 0.7356
Epoch 63/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5497 - acc: 0.7354
Epoch 64/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5498 - acc: 0.7346
Epoch 65/100
25376/25724 [============================>.] - ETA: 0s - loss: 0.5489 - acc: 0.7353
Epoch 00065: saving model to optimized_checkpoints_second/weights.65.hdf5
25724/25724 [==============================] - 1s 25us/sample - loss: 0.5493 - acc: 0.7348
Epoch 66/100
25724/25724 [==============================] - 1s 24us/sample - loss: 0.5494 - acc: 0.7356
Epoch 67/100
25724/25724 [==============================] - 1s 24us/sample - loss: 0.5492 - acc: 0.73440s - loss: 0.5536 -
Epoch 68/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5495 - acc: 0.7357
Epoc

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5599 - acc: 0.7291
Loss: 0.559891555218933, Accuracy: 0.7290962338447571


In [41]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharityOptimized_Second.h5")

In [42]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=54))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [43]:
# Add model weight checkpoints during training
import os
from tensorflow.keras.callbacks import ModelCheckpoint
os.makedirs("optimized_checkpoints_second/", exist_ok=True)
checkpoint_path="optimized_checkpoints_second/weights.{epoch}.hdf5"

# Create callback that saves the model's weights every 5 epochs
cp_callback = ModelCheckpoint(filepath=checkpoint_path,
                              verbose=1,
                              save_weights_only=True,
                              save_freq="epoch",
                              period=5)

In [44]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100, callbacks=[cp_callback])

Epoch 1/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5484 - acc: 0.7370
Epoch 2/100
25724/25724 [==============================] - 1s 28us/sample - loss: 0.5480 - acc: 0.7356
Epoch 3/100
25724/25724 [==============================] - 1s 24us/sample - loss: 0.5486 - acc: 0.7363
Epoch 4/100
25724/25724 [==============================] - 1s 24us/sample - loss: 0.5481 - acc: 0.7359
Epoch 5/100
24256/25724 [===========================>..] - ETA: 0s - loss: 0.5477 - acc: 0.7361
Epoch 00005: saving model to optimized_checkpoints_second/weights.5.hdf5
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5483 - acc: 0.7361
Epoch 6/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5480 - acc: 0.7374
Epoch 7/100
25724/25724 [==============================] - 1s 22us/sample - loss: 0.5482 - acc: 0.7357
Epoch 8/100
25724/25724 [==============================] - 1s 24us/sample - loss: 0.5481 - acc: 0.7365
Epoch 9/100
25724/2

Epoch 61/100
25724/25724 [==============================] - 1s 24us/sample - loss: 0.5454 - acc: 0.7365
Epoch 62/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5451 - acc: 0.7355
Epoch 63/100
25724/25724 [==============================] - 1s 24us/sample - loss: 0.5450 - acc: 0.7364
Epoch 64/100
25724/25724 [==============================] - 1s 25us/sample - loss: 0.5454 - acc: 0.7374
Epoch 65/100
24960/25724 [============================>.] - ETA: 0s - loss: 0.5454 - acc: 0.7365
Epoch 00065: saving model to optimized_checkpoints_second/weights.65.hdf5
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5452 - acc: 0.7369
Epoch 66/100
25724/25724 [==============================] - 1s 24us/sample - loss: 0.5452 - acc: 0.7372
Epoch 67/100
25724/25724 [==============================] - 1s 23us/sample - loss: 0.5454 - acc: 0.7362
Epoch 68/100
25724/25724 [==============================] - 1s 26us/sample - loss: 0.5447 - acc: 0.7352
Epoch 69/1

In [45]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5602 - acc: 0.7308
Loss: 0.5602151941210466, Accuracy: 0.7308454513549805


In [46]:
!pip install keras-tuner

In [49]:
# Import the kerastuner library
# import kerastuner as kt
import keras_tuner as kt
import os

tuner = kt.Hyperband(
    create_model,
    objective="val_acc",
    max_epochs=20,
    directory=os.path.normpath('C:/AlphabetSoupCharity_Optimization_Second'),
    hyperband_iterations=54)

INFO:tensorflow:Reloading Oracle from existing project C:\AlphabetSoupCharity_Optimization_Second\untitled_project\oracle.json


In [50]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 61 Complete [00h 00m 17s]
val_acc: 0.7335277199745178

Best val_acc So Far: 0.7344606518745422
Total elapsed time: 00h 06m 29s
INFO:tensorflow:Oracle triggered exit


In [51]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'sigmoid',
 'first_units': 9,
 'num_layers': 4,
 'units_0': 5,
 'units_1': 5,
 'units_2': 9,
 'units_3': 7,
 'units_4': 5,
 'units_5': 3,
 'tuner/epochs': 3,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 2,
 'tuner/round': 0}

In [52]:
best_model = tuner.get_best_models(1)[0]
best_model.save('AlphabetSoupCharity_Optimization_Second.h5')